# 📤 Entregar Trabajo

**Instrucciones:**
1. Completa todos los ejercicios en este notebook
2. Ejecuta todas las celdas
3. Ejecuta las 2 celdas de abajo para entregar

**NO requiere cuenta ni login** - solo ingresa tu nombre y listo.

In [ ]:
import json
import asyncio

# Import browser APIs (only available in JupyterLite)
try:
    from js import fetch, document, Object, JSON
    from pyodide.ffi import to_js
    print("✅ Browser APIs cargadas correctamente")
except ImportError as e:
    print(f"⚠️ Este notebook debe ejecutarse en JupyterLite (navegador)")
    print(f"   Visita: https://juakongdf.github.io/chaos-lab/")
    raise

SERVER_URL = "https://chaos-sub.vercel.app/submit"

async def submit_current_notebook():
    try:
        student_name = document.getElementById("studentName").value.strip()
        if not student_name:
            document.getElementById("result").innerHTML = '<div style="color:red">❌ Ingresa tu nombre</div>'
            return
        
        document.getElementById("result").innerHTML = '<div>⏳ Enviando...</div>'
        
        with open("99_Entregar_Trabajo.ipynb", 'r') as f:
            notebook_content = json.load(f)
        
        payload = {"student_name": student_name, "notebook": notebook_content}
        js_payload = to_js(payload, dict_converter=Object.fromEntries)
        fetch_options = {
            "method": "POST",
            "headers": {"Content-Type": "application/json"},
            "body": JSON.stringify(js_payload)
        }
        
        response = await fetch(SERVER_URL, to_js(fetch_options, dict_converter=Object.fromEntries))
        response_text = await response.text()
        
        if response.ok:
            result_data = json.loads(response_text)
            document.getElementById("result").innerHTML = f'<div style="color:green">✅ ¡Entregado! Archivo: {result_data.get("filename")}</div>'
        else:
            document.getElementById("result").innerHTML = f'<div style="color:red">❌ Error: {response_text}</div>'
    except Exception as e:
        document.getElementById("result").innerHTML = f'<div style="color:red">❌ {str(e)}</div>'

print("✅ Sistema configurado - Ejecuta la celda de abajo para mostrar el formulario")

In [ ]:
from IPython.display import display, HTML

form = """
<div style="padding:20px; background:#f0f0f0; border-radius:8px; max-width:500px; margin:20px auto;">
    <h3>📝 Entregar Trabajo</h3>
    <input id="studentName" placeholder="Tu nombre completo" style="width:100%; padding:10px; font-size:16px; margin:10px 0;"/>
    <button onclick="submitWork()" style="width:100%; padding:12px; background:#4CAF50; color:white; border:none; border-radius:4px; font-size:16px; cursor:pointer;">🚀 Entregar</button>
    <div id="result" style="margin-top:15px;"></div>
</div>
<script>
async function submitWork() {
    await pyodide.runPythonAsync('import asyncio; asyncio.ensure_future(submit_current_notebook())');
}
</script>
"""
display(HTML(form))